## Crank-Nicolson scheme

Implicit method used with parabolic PDEs

Main idea: Take the average between the solutions at $t^n$ and $t^{n+1}$ in evaluation of the spatial derivative.  This is so that the time derivative will be discretized with a centered scheme, thus giving it second order accuracy.

1D Heat Equation:

$$\frac{\partial T}{\partial t} = \alpha \frac{\partial ^2 T}{\partial x^2}$$

From this, Crank-Nicolson scheme leads to:

$$\frac{T_i ^{n+1} - T_i ^n}{\Delta t} = \alpha \circ \frac{1}{2} \left( \frac{T_{i+1} ^{n+1} - 2T_i ^{n+1} + T_{i-1} ^{n+1}}{\Delta x^2} + \frac{T_{i+1} ^n - 2T_i ^n + T_{i-1} ^n}{\Delta x^2} \right)$$

n and n+1 both appear on the right hand side.  Gotta rearrange the equation.

Everything we don't know is on the left and everything we know is on the right.

$$-T_{i-1} ^{n+1} + 2 \left( \frac{\Delta x^2}{\alpha \Delta t} + 1 \right) T_i ^{n+1} - T_{i+1} ^{n+1} = T_{i-1} ^n + 2 \left( \frac{\Delta x^2}{\alpha \Delta t} - 1 \right) T_i ^n + T_{i+1} ^n$$

LHS looks like the matrix from notebook 2.  Not much change for the boundary conditions either.

### The linear system

Need to solve a linear system on every time step of the form:

$$[A][T_{int} ^{n+1}] = [b] + [b]_{b.c.}$$

Need a function that will create the coefficient matrix and the RHS for the heat conduction problem from notebook 2, with a Dirichlet BC at $x = 0$ and zero-flux boundary $(q = 0)$ at $x = L$